In [1]:
from binance.client import Client
from config.secret import *
import pandas as pd
import time
import os

In [2]:
client = Client(API_KEY, API_SECRET)

In [3]:
def get_all_symbols_binance():
    symbols = []
    tickers = client.get_orderbook_tickers()
    for ticker in tickers:
        symbols.append(ticker['symbol'])
    symbols = [symbol for symbol in symbols if symbol.endswith("USDT")]
    return symbols

In [4]:
symbols = get_all_symbols_binance()

In [5]:
def data_handler(data):
    column_names = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume',
                'number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore']
    df = pd.DataFrame(data, columns=column_names)
    df['date_open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['date_close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    return df

In [5]:
available_symbols = [symbol.replace('.parquet', '') for symbol in os.listdir('data/1d_data')]
for symbol in available_symbols:
    print(f'- {symbol}')

- ADAUSDT
- BNBUSDT
- BTCUSDT
- EOSUSDT
- ETCUSDT
- ETHUSDT
- ICXUSDT
- IOTAUSDT
- LTCUSDT
- NEOUSDT
- NULSUSDT
- ONTUSDT
- PAXUSDT
- QTUMUSDT
- TRXUSDT
- TUSDUSDT
- USDCUSDT
- VETUSDT
- XLMUSDT
- XRPUSDT


In [16]:
start_time = '1 Jan, 2019'
end_time = '1 Mar, 2021'
interval = '1m'
path = f'data/{interval}_data/'

In [17]:
for symbol in available_symbols:
    try:
        data = client.get_historical_klines(symbol, interval, start_time, end_time)
        df = data_handler(data)
        df.to_parquet(f'{path}{symbol}.parquet')
        time.sleep(0.1)
    except Exception as e:
        print(e)